### Stock Data Model

In [153]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#### Load Data without sentiment and split into test and training dataset

In [4]:
df = pd.read_csv('Stock Data + Sentiment Final.csv')
df_base = pd.read_csv('Stock_Data.csv')

In [14]:
df['title_sentiment'] = [1 if score > 0.05 
                             else -1 if score < -0.05
                             else 0 
                             for score in df['title sentiment score']]
df['content_sentiment'] = [1 if score > 0.05 
                             else -1 if score < -0.05
                             else 0 
                             for score in df['content sentiment score']]

In [ ]:
df.set_index(['Date', 'Exchange_Name'], inplace = True)
df.head()

#### Removes dates where stock market wasn't open (i.e weekends, public holidays). Inner join occured, therefore content sentiment score can be indexed.

In [49]:
df.Date.nunique()

583

In [5]:
df.drop(df[df['content sentiment score'] == 0].index, inplace = True)

In [51]:
df.Date.nunique()

581

##### Two values removed

### Addition Exploration

#### Add binary buy sell condition.

In [12]:
df['buy/sell'] = np.where(df['Close'] > df['Open'], 1, 0)
df_base['buy/sell'] = np.where(df_base['Close'] > df_base['Open'], 1, 0)

In [15]:
df.head()

,Date,content sentiment score,title sentiment score,Exchange_Name,Adj Close,Close,High,Low,Open,Volume,buy/sell,title_sentiment,content_sentiment
0,2015-01-02,0.07875,-0.01125,Dow Jones,17832.990234,17832.990234,17951.779297,17731.300781,17823.070313,76270000,1,0,1
1,2015-01-02,0.07875,-0.01125,NYSE,10830.919922,10830.919922,10889.250000,10770.509766,10859.799805,2708700000,0,0,1
2,2015-01-02,0.07875,-0.01125,TSX/S&P,14753.700195,14753.700195,14756.299805,14631.400391,14637.299805,132965800,1,0,1
3,2015-01-02,0.07875,-0.01125,NASDAQ,4726.810059,4726.810059,4777.009766,4698.109863,4760.240234,1435150000,0,0,1
4,2015-01-02,0.07875,-0.01125,S&P,2058.199951,2058.199951,2072.360107,2046.040039,2058.899902,2708700000,0,0,1


In [16]:
df_base.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Exchange_Name,buy/sell
0,2015-01-02,17823.070313,17951.779297,17731.300781,17832.990234,17832.990234,76270000,Dow Jones,1
1,2015-01-02,10859.799805,10889.250000,10770.509766,10830.919922,10830.919922,2708700000,NYSE,0
2,2015-01-02,14637.299805,14756.299805,14631.400391,14753.700195,14753.700195,132965800,TSX/S&P,1
3,2015-01-02,4760.240234,4777.009766,4698.109863,4726.810059,4726.810059,1435150000,NASDAQ,0
4,2015-01-02,2058.899902,2072.360107,2046.040039,2058.199951,2058.199951,2708700000,S&P,0


In [18]:
stockdf = df_base

In [27]:
stockdf.set_index(['Date'], inplace = True)

#### Encode Stock Markets

In [19]:
stockdf.replace({'Dow Jones': 1, 'NYSE' : 2 , 'TSX/S&P' : 3, 'NASDAQ' : 4 , 'S&P' : 5}, inplace = True)

In [20]:
df2 = df
df2.replace({'Dow Jones': 1, 'NYSE' : 2 , 'TSX/S&P' : 3, 'NASDAQ' : 4 , 'S&P' : 5}, inplace = True)

In [28]:
df2.set_index(['Date'], inplace = True)

In [29]:
stock1_train, stock1_test, sent_stock1_train, sent_stock1_test = train_test_split(stockdf,
                                                                              stockdf['buy/sell'],
                                                                             test_size = 0.3, random_state = 0)

In [30]:
stock_train, stock_test, sent_stock_train, sent_stock_test = train_test_split(df2,
                                                                              df2['buy/sell'],
                                                                             test_size = 0.3, random_state = 0)

#### Run model without sentiment scores or weighting

In [25]:
from sklearn.linear_model import LogisticRegression

In [32]:
classifier = LogisticRegression()
classifier.fit(stock1_train, sent_stock1_train)
sent_pred = classifier.predict(stock1_test)

In [33]:
print(confusion_matrix(sent_stock1_test,sent_pred))
print(classification_report(sent_stock1_test,sent_pred))
print(accuracy_score(sent_stock1_test, sent_pred))

[[  0 515]
 [  0 617]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       515
           1       0.55      1.00      0.71       617

    accuracy                           0.55      1132
   macro avg       0.27      0.50      0.35      1132
weighted avg       0.30      0.55      0.38      1132

0.5450530035335689


C:\Users\wrigh\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [54]:
classifier = KNeighborsClassifier(n_neighbors = 5, weights = 'distance')
classifier.fit(stock1_train, sent_stock1_train)
sent_pred = classifier.predict(stock1_test)

In [55]:
print(confusion_matrix(sent_stock1_test,sent_pred))
print(classification_report(sent_stock1_test,sent_pred))
print(accuracy_score(sent_stock1_test, sent_pred))

[[289 226]
 [219 398]]
              precision    recall  f1-score   support

           0       0.57      0.56      0.57       515
           1       0.64      0.65      0.64       617

    accuracy                           0.61      1132
   macro avg       0.60      0.60      0.60      1132
weighted avg       0.61      0.61      0.61      1132

0.6068904593639576


##### Overfitting, not a large sample size. No Classification of sell.

### Run model with sentiment scores (both summed and averaged) - Log Reg and KNN

#### Average

In [34]:
classifier = LogisticRegression()
classifier.fit(stock_train, sent_stock_train)
sent_pred = classifier.predict(stock_test)

In [35]:
print(confusion_matrix(sent_stock_test,sent_pred))
print(classification_report(sent_stock_test,sent_pred))
print(accuracy_score(sent_stock_test, sent_pred))

[[  0 407]
 [  0 453]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       407
           1       0.53      1.00      0.69       453

    accuracy                           0.53       860
   macro avg       0.26      0.50      0.35       860
weighted avg       0.28      0.53      0.36       860

0.5267441860465116


C:\Users\wrigh\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
from sklearn.neighbors import KNeighborsClassifier

In [57]:
classifier = KNeighborsClassifier(n_neighbors = 5, weights = 'distance')
classifier.fit(stock_train, sent_stock_train)
sent_pred = classifier.predict(stock_test)

In [58]:
print(confusion_matrix(sent_stock_test,sent_pred))
print(classification_report(sent_stock_test,sent_pred))
print(accuracy_score(sent_stock_test, sent_pred))

[[255 152]
 [174 279]]
              precision    recall  f1-score   support

           0       0.59      0.63      0.61       407
           1       0.65      0.62      0.63       453

    accuracy                           0.62       860
   macro avg       0.62      0.62      0.62       860
weighted avg       0.62      0.62      0.62       860

0.6209302325581395


#### Summed

In [126]:
df_sum = pd.read_csv('Stock Data + Sentiment.csv')
df_sum = pd.read_csv('Stock Data + Sentiment.csv')
df_sum.drop(['Number of Records', 'date', 'Adj Close'], axis = 1, inplace = True)
df_sum.set_index(['Date'], inplace = True)
df_sum['title_sentiment'] = [1 if score > 0.05 
                             else -1 if score < -0.05
                             else 0 
                             for score in df_sum['title sentiment score']]
df_sum['content_sentiment'] = [1 if score > 0.05 
                             else -1 if score < -0.05
                             else 0 
                             for score in df_sum['content sentiment score']]
df_sum['buy/sell'] = np.where(df_sum['Close'] > df_sum['Open'], 1, 0)
df_sum.replace({'Dow Jones': 1, 'NYSE' : 2 , 'TSX/S&P' : 3, 'NASDAQ' : 4 , 'S&P' : 5}, inplace = True)
df_sum.dropna(inplace = True)

In [127]:
df_sum.drop(df_sum[df_sum['content sentiment score'] == 0].index, inplace = True)

In [128]:
df_sum.head(15)

,Close,Exchange_Name,High,Low,Open,Volume,content sentiment score,title sentiment score,title_sentiment,content_sentiment,buy/sell
Date,,,,,,,,,,,
2015-01-02,17832.990234,1,17951.779297,17731.300781,17823.070313,76270000,0.945,-0.135,-1,1,1
2015-01-02,10830.919922,2,10889.250000,10770.509766,10859.799805,2708700000,0.945,-0.135,-1,1,0
2015-01-02,14753.700195,3,14756.299805,14631.400391,14637.299805,132965800,0.945,-0.135,-1,1,1
2015-01-02,4726.810059,4,4777.009766,4698.109863,4760.240234,1435150000,0.945,-0.135,-1,1,0
2015-01-02,2058.199951,5,2072.360107,2046.040039,2058.899902,2708700000,0.945,-0.135,-1,1,0
2015-01-06,4592.740234,4,4667.330078,4567.589844,4666.850098,2167320000,0.205,-0.067,-1,1,0
2015-01-06,10514.870117,2,10647.209961,10457.889648,10610.780273,4460110000,0.205,-0.067,-1,1,0
2015-01-06,2002.609985,5,2030.250000,1992.439941,2022.150024,4460110000,0.205,-0.067,-1,1,0
2015-01-06,14246.799805,3,14370.400391,14162.000000,14368.099609,273374500,0.205,-0.067,-1,1,0


In [129]:
df_sum.dtypes
df_y = df_sum['buy/sell']

In [132]:
df_sum.drop('buy/sell', axis = 1, inplace = True)

In [133]:
xsum_train, xsum_test, ysum_train, ysum_test = train_test_split(df_sum,df_y,
                                                                test_size = 0.3, random_state = 0)

In [137]:
classifier = LogisticRegression()
classifier.fit(xsum_train, ysum_train)
ysum_pred = classifier.predict(xsum_test)

In [138]:
print(confusion_matrix(ysum_test, ysum_pred))
print(classification_report(ysum_test,ysum_pred))
print(accuracy_score(ysum_test, ysum_pred))

[[  0 405]
 [  0 452]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       405
           1       0.53      1.00      0.69       452

    accuracy                           0.53       857
   macro avg       0.26      0.50      0.35       857
weighted avg       0.28      0.53      0.36       857

0.5274212368728122


C:\Users\wrigh\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [156]:
classifier = KNeighborsClassifier(n_neighbors = 5, weights= 'distance')
classifier.fit(xsum_train, ysum_train)
ysum_pred = classifier.predict(xsum_test)

In [158]:
print(confusion_matrix(ysum_test, ysum_pred))
print(classification_report(ysum_test,ysum_pred))
print(accuracy_score(ysum_test, ysum_pred))

[[185 220]
 [176 276]]
              precision    recall  f1-score   support

           0       0.51      0.46      0.48       405
           1       0.56      0.61      0.58       452

    accuracy                           0.54       857
   macro avg       0.53      0.53      0.53       857
weighted avg       0.54      0.54      0.54       857

0.5379229871645275


#### Scale Data

In [121]:
from sklearn.preprocessing import StandardScaler

In [ ]:
df[df.columns] = scaler.fit_transform(df[df.columns])

In [139]:
x = df_sum.values
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
df_sum = pd.DataFrame(x_scaled)
df_sum.head()

,0,1,2,3,4,5,6,7,8,9
0,1.265062,-1.414214,1.273253,1.261588,1.263904,-1.127110,-1.257987,-0.958717,-2.429782,0.141875
1,0.101252,-0.707107,0.103714,0.099740,0.106377,0.467639,-1.257987,-0.958717,-2.429782,0.141875
2,0.753255,0.000000,0.744089,0.744174,0.734323,-1.092763,-1.257987,-0.958717,-2.429782,0.141875
3,-0.913308,0.707107,-0.908458,-0.913823,-0.907573,-0.303888,-1.257987,-0.958717,-2.429782,0.141875
4,-1.356856,1.414214,-1.356342,-1.356488,-1.356626,0.467639,-1.257987,-0.958717,-2.429782,0.141875


In [140]:
xsum_train, xsum_test, ysum_train, ysum_test = train_test_split(df_sum,df_y,
                                                                test_size = 0.3, random_state = 0)

In [150]:
classifier = KNeighborsClassifier(n_neighbors = 7)
classifier.fit(xsum_train, ysum_train)
ysum_pred = classifier.predict(xsum_test)

In [151]:
print(confusion_matrix(ysum_test, ysum_pred))
print(classification_report(ysum_test,ysum_pred))

[[185 220]
 [181 271]]
              precision    recall  f1-score   support

           0       0.51      0.46      0.48       405
           1       0.55      0.60      0.57       452

    accuracy                           0.53       857
   macro avg       0.53      0.53      0.53       857
weighted avg       0.53      0.53      0.53       857



In [152]:
accuracy_score(ysum_test, ysum_pred)

0.5320886814469078

In [ ]:
classifier = KNeighborsClassifier(n_neighbors = 7, weights= 'distance')
classifier.fit(xsum_train, ysum_train)
ysum_pred = classifier.predict(xsum_test)

print(confusion_matrix(ysum_test, ysum_pred))
print(classification_report(ysum_test,ysum_pred))
print(accuracy_score(ysum_test, ysum_pred)

##### Similar accuracy

#### Oversample - due to a value count of 4 for negative sentiment, undersampling will not occur.

In [29]:
print(sent_stock_train.value_counts())
df_neutral_s = stock_train[sent_stock_train == 0]
df_positive_s = stock_train[sent_stock_train == 1]
df_negative_s = stock_train[sent_stock_train == -1]

 1    1899
 0      95
-1       4
Name: content_sentiment, dtype: int64


In [30]:
count_positive_s = 1899
count_neutral_s = 95
count_negative_s = 4

In [31]:
over_neg_s = df_negative_s.sample(count_positive_s, replace=True)
over_neut_s = df_neutral_s.sample(count_positive_s, replace = True)
over_train_s = pd.concat([df_positive_s, over_neg_s, over_neut_s], axis=0)
print(over_train_s['content_sentiment'].value_counts())

-1    1899
 1    1899
 0    1899
Name: content_sentiment, dtype: int64


#### Run Model again.

In [293]:
classifier = LogisticRegression()
classifier.fit(over_train_s, over_train_s['content_sentiment'])
sent_pred = classifier.predict(stock_test)

In [294]:
print(confusion_matrix(sent_stock_test,sent_pred))
print(classification_report(sent_stock_test,sent_pred))
print(accuracy_score(sent_stock_test, sent_pred))

[[  0   0   1]
 [  0   0  39]
 [  1   0 816]]
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         1
           0       0.00      0.00      0.00        39
           1       0.95      1.00      0.98       817

    accuracy                           0.95       857
   macro avg       0.32      0.33      0.33       857
weighted avg       0.91      0.95      0.93       857

0.9521586931155193


C:\Users\wrigh\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### KNN Model

In [34]:
classifier = KNeighborsClassifier()
classifier.fit(over_train_s, over_train_s['content_sentiment'])
sent_pred = classifier.predict(stock_test)

In [37]:
print(confusion_matrix(sent_stock_test,sent_pred))
print(classification_report(sent_stock_test,sent_pred))
print(accuracy_score(sent_stock_test, sent_pred))

[[  1   0   0]
 [  0  15  24]
 [  4  83 730]]
              precision    recall  f1-score   support

          -1       0.20      1.00      0.33         1
           0       0.15      0.38      0.22        39
           1       0.97      0.89      0.93       817

    accuracy                           0.87       857
   macro avg       0.44      0.76      0.49       857
weighted avg       0.93      0.87      0.90       857

0.8704784130688448


#### The presence of a single negative in the test set is  cause for concern, therefore a resamample will be done on the whole datatset and the model will be tested again.

In [100]:
print(df2['content_sentiment'].value_counts())
df_neutral = df2[df2['content_sentiment'] == 0]
df_p = df2[df2['content_sentiment'] == 1]
df_n = df2[df2['content_sentiment'] == -1]

 1    2716
 0     134
-1       5
Name: content_sentiment, dtype: int64


In [101]:
count_positive_df2 = 2716
count_neutral_df2 = 134
count_negative_df2 = 5

#### Oversample - due to a value count of 4 for negative sentiment, undersampling will not occur.

In [102]:
over_neg_df2 = df_n.sample(count_positive_df2, replace=True)
over_neut_df2 = df_neutral.sample(count_positive_df2, replace = True)
df2_over = pd.concat([df_p, over_neg_df2, over_neut_df2], axis=0)
print(df2_over['content_sentiment'].value_counts())

-1    2716
 1    2716
 0    2716
Name: content_sentiment, dtype: int64


#### Split oversample into training and test sets.

In [103]:
df2_over.head()
df2_over.dtypes

content sentiment score    float64
title sentiment score      float64
Exchange_Name                int64
Adj Close                  float64
Close                      float64
High                       float64
Low                        float64
Open                       float64
Volume                       int64
title_sentiment              int64
content_sentiment            int64
dtype: object

In [331]:
x_train, x_test, y_train, y_test = train_test_split(df2_over, df2_over['content_sentiment'], 
                                                    test_size = 0.2, random_state = 0)

#### Run Oversampling Model

In [332]:
classifier = LogisticRegression()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)

In [333]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[  0   0 538]
 [  0   0 558]
 [  0   0 534]]
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       538
           0       0.00      0.00      0.00       558
           1       0.33      1.00      0.49       534

    accuracy                           0.33      1630
   macro avg       0.11      0.33      0.16      1630
weighted avg       0.11      0.33      0.16      1630

0.3276073619631902


C:\Users\wrigh\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 32% accuaracy with oversampling entire dataset. Only positive values being predicted correctly. 

#### K Nearest Neighbours 

In [334]:
classifier = KNeighborsClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)

In [335]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[538   0   0]
 [  0 558   0]
 [  2  71 461]]
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00       538
           0       0.89      1.00      0.94       558
           1       1.00      0.86      0.93       534

    accuracy                           0.96      1630
   macro avg       0.96      0.95      0.95      1630
weighted avg       0.96      0.96      0.95      1630

0.9552147239263804


#### Adjust n and weighting

In [326]:
classifier = KNeighborsClassifier(n_neighbors=11)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)

In [327]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[538   0   0]
 [  0 558   0]
 [  3 114 417]]
              precision    recall  f1-score   support

          -1       0.99      1.00      1.00       538
           0       0.83      1.00      0.91       558
           1       1.00      0.78      0.88       534

    accuracy                           0.93      1630
   macro avg       0.94      0.93      0.93      1630
weighted avg       0.94      0.93      0.93      1630

0.9282208588957055


#### Opposite error occurs, with neutral and negative values being predicted positive 100% of the time. 

#### Normalize data

In [61]:
from sklearn import preprocessing
from sklearn import utils

In [109]:
x = df2.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_norm = pd.DataFrame(x_scaled)
df_norm.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.413068,0.555469,0.00,0.812385,0.812385,0.816848,0.810069,0.812159,0.010039,0.5,1.0
1,0.413068,0.555469,0.25,0.456948,0.456948,0.458631,0.455904,0.458476,0.356528,0.5,1.0
2,0.413068,0.555469,0.50,0.656075,0.656075,0.654771,0.652346,0.650345,0.017501,0.5,1.0
3,0.413068,0.555469,0.75,0.147094,0.147094,0.148613,0.146942,0.148662,0.188899,0.5,1.0
4,0.413068,0.555469,1.00,0.011631,0.011631,0.011430,0.012005,0.011454,0.356528,0.5,1.0


In [110]:
df_norm.head()
df_norm.rename(columns = { 10 : 'sent', 1 : 'Ex_Name', 9 : 't_sent'}, inplace = True)
df_norm.head()

,0,Ex_Name,2,3,4,5,6,7,8,t_sent,sent
0,0.413068,0.555469,0.00,0.812385,0.812385,0.816848,0.810069,0.812159,0.010039,0.5,1.0
1,0.413068,0.555469,0.25,0.456948,0.456948,0.458631,0.455904,0.458476,0.356528,0.5,1.0
2,0.413068,0.555469,0.50,0.656075,0.656075,0.654771,0.652346,0.650345,0.017501,0.5,1.0
3,0.413068,0.555469,0.75,0.147094,0.147094,0.148613,0.146942,0.148662,0.188899,0.5,1.0
4,0.413068,0.555469,1.00,0.011631,0.011631,0.011430,0.012005,0.011454,0.356528,0.5,1.0


In [113]:
df_norm.dtypes
df_norm['Ex_Name'] = df_norm['Ex_Name'].astype('int64')
df_norm['t_sent'] = df_norm['t_sent'].astype('int64')
df_norm['sent'] = df_norm['sent'].astype('int64')
df_norm.dtypes

0          float64
Ex_Name      int64
2          float64
3          float64
4          float64
5          float64
6          float64
7          float64
8          float64
t_sent       int64
sent         int64
dtype: object

In [114]:
xn_train, xn_test, yn_train, yn_test = train_test_split(df_norm, df_norm['sent'], 
                                                        test_size = 0.3, random_state = 0)

In [115]:
classifier = KNeighborsClassifier()
classifier.fit(xn_train, yn_train)
yn_pred = classifier.predict(xn_test)

In [116]:
print(confusion_matrix(yn_test,yn_pred))
print(classification_report(yn_test,yn_pred))
print(accuracy_score(yn_test, yn_pred))

[[ 40   0]
 [  0 817]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       1.00      1.00      1.00       817

    accuracy                           1.00       857
   macro avg       1.00      1.00      1.00       857
weighted avg       1.00      1.00      1.00       857

1.0


In [117]:
classifier = LogisticRegression()
classifier.fit(xn_train, yn_train)
y_pred = classifier.predict(xn_test)

In [118]:
print(confusion_matrix(yn_test,yn_pred))
print(classification_report(yn_test,yn_pred))
print(accuracy_score(yn_test, yn_pred))

[[ 40   0]
 [  0 817]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       1.00      1.00      1.00       817

    accuracy                           1.00       857
   macro avg       1.00      1.00      1.00       857
weighted avg       1.00      1.00      1.00       857

1.0


#### Shift to Keras to build Neural Network

In [155]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

Using TensorFlow backend.
